In [1]:
## Packages 
import csv
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm
import pickle

# For graphs
import networkx as nx 
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, HinSAGE, link_classification
from stellargraph import globalvar

# For deep learning
from tensorflow import keras 
import tensorflow as tf

'''
config = tf.ConfigProto(intra_op_parallelism_threads=8, inter_op_parallelism_threads=2, allow_soft_placement=True, device_count = {'CPU': 8})

session = tf.Session(config=config)

os.environ["OMP_NUM_THREADS"] = "8"

os.environ["KMP_BLOCKTIME"] = "30"

os.environ["KMP_SETTINGS"] = "1"

os.environ["KMP_AFFINITY"]= "granularity=fine,verbose,compact,1,0"
'''

# For processing node texts
from sklearn.feature_extraction import text as fe

# Dimensionality reduction
from sklearn.decomposition import NMF, LatentDirichletAllocation

# Word embeddings
import gensim 
from gensim.models import Word2Vec

# For stemming
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize

In [2]:
'''
tf.test.gpu_device_name()
#!cat /proc/meminfo
!cat /proc/cpuinfo
'''

'\ntf.test.gpu_device_name()\n#!cat /proc/meminfo\n!cat /proc/cpuinfo\n'

In [3]:
# Defining paths
small_matrix_path = r"pickles/stemmed_lda_64_matrix.PICKLE"
corpus_path = r"pickles/stemmed_corpus.PICKLE"

if os.path.exists(small_matrix_path):
    with open(small_matrix_path, 'rb') as f:
        lda_matrix = pickle.load(f)
    f.close()
else:
    print('Error')

In [4]:
# Storing in dataframe
n1, n2 = lda_matrix.shape
# Creating feature names
feature_names = ["w_{}".format(ii) for ii in range(n2)]
ids = sorted(range(n1), key=str)
node_data = pd.DataFrame(data=lda_matrix, index=ids, columns=feature_names)
print(node_data.shape)

(33226, 64)


In [5]:
print(node_data)

           w_0       w_1       w_2       w_3       w_4       w_5       w_6  \
0     0.000051  0.000051  0.000051  0.000051  0.000051  0.111335  0.000051   
1     0.000008  0.000008  0.000008  0.000008  0.000008  0.000008  0.000008   
10    0.000011  0.017704  0.000011  0.000011  0.000011  0.000011  0.000011   
100   0.155756  0.000953  0.000001  0.000001  0.000001  0.003889  0.000001   
1000  0.000049  0.000049  0.000049  0.000049  0.000049  0.016454  0.000049   
...        ...       ...       ...       ...       ...       ...       ...   
9995  0.000081  0.000081  0.000081  0.000081  0.000081  0.000081  0.000081   
9996  0.000009  0.000009  0.000009  0.000009  0.000009  0.000009  0.000009   
9997  0.000005  0.000005  0.000005  0.000005  0.000005  0.008869  0.000005   
9998  0.000009  0.000009  0.000009  0.000009  0.000009  0.014683  0.000009   
9999  0.001042  0.001042  0.001042  0.001042  0.001042  0.403546  0.001042   

           w_7       w_8       w_9  ...      w_54      w_55    

## Make Graphs Great Again

Time to get training and test data

In [6]:
with open(r"training.txt", "r") as f:
    reader = csv.reader(f)
    training  = list(reader)
# in order of training examples
training = [element[0].split(" ") for element in training]
training = pd.DataFrame(training, columns=['Node1', 'Node2', 'Link'])
print("Training examples shape: {}".format(training.shape))

with open(r"testing.txt", "r") as f:
    reader = csv.reader(f)
    testing  = list(reader)
# in order of testing examples
testing = [element[0].split(" ") for element in testing]
testing = pd.DataFrame(testing, columns=['Node1', 'Node2'])
print("Testing examples shape: {}".format(testing.shape))

Training examples shape: (453797, 3)
Testing examples shape: (113450, 2)


In [7]:
linked_nodes = training.loc[training['Link']=='1']
linked_nodes = linked_nodes[['Node1', 'Node2']]
linked_nodes.to_csv('linked_nodes.txt', sep=' ', index=False, header=False)

In [8]:
# Read edges and create NetworkX graph
edgelist = pd.read_csv("linked_nodes.txt", sep=' ', header=None, names=["source", "target"])
edgelist["label"] = "cites"  # set the edge type
G_all_nx = nx.from_pandas_edgelist(edgelist, edge_attr="label")
G_all_nx.add_nodes_from(ids)
nx.set_node_attributes(G_all_nx, "paper", "label")

# Initialize Stellargraph with node features of text
#G_all = sg.StellarGraph(G_all_nx, node_features=node_data[feature_names])

# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(G_all_nx)

# Randomly sample a fraction p=0.1 of all positive links, 
# and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=True)

# Define an edge splitter on the reduced graph G_test:
edge_splitter_train = EdgeSplitter(G_test)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=True)

G_test = sg.StellarGraph(G_test, node_features=node_data[feature_names])
G_train = sg.StellarGraph(G_train, node_features=node_data[feature_names])

Removed 1000 edges
Removed 2000 edges
Removed 3000 edges
Removed 4000 edges
Removed 5000 edges
Removed 6000 edges
Removed 7000 edges
Removed 8000 edges
Removed 9000 edges
Removed 10000 edges
Removed 11000 edges
Removed 12000 edges
Removed 13000 edges
Removed 14000 edges
Removed 15000 edges
Removed 16000 edges
Removed 17000 edges
Removed 18000 edges
Removed 19000 edges
Removed 20000 edges
Removed 21000 edges
Removed 22000 edges
Removed 23000 edges
Removed 24000 edges
Removed 25000 edges
Removed 26000 edges
Removed 27000 edges
Removed 28000 edges
Sampled 1000 negative examples
Sampled 2000 negative examples
Sampled 3000 negative examples
Sampled 4000 negative examples
Sampled 5000 negative examples
Sampled 6000 negative examples
Sampled 7000 negative examples
Sampled 8000 negative examples
Sampled 9000 negative examples
Sampled 10000 negative examples
Sampled 11000 negative examples
Sampled 12000 negative examples
Sampled 13000 negative examples
Sampled 14000 negative examples
Sampled 15

In [9]:
print(G_test.info())
print(G_train.info())

StellarGraph: Undirected multigraph
 Nodes: 33226, Edges: 255261

 Node types:
  paper: [33226]
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [255261]

StellarGraph: Undirected multigraph
 Nodes: 33226, Edges: 229735

 Node types:
  paper: [33226]
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [229735]



In [24]:
batch_size = 20
epochs = 5
num_samples = [200, 200]

train_gen = GraphSAGELinkGenerator(G_train, batch_size, num_samples).flow(
    edge_ids_train, edge_labels_train, shuffle=True)
test_gen = GraphSAGELinkGenerator(G_test,  batch_size, num_samples).flow(
    edge_ids_test, edge_labels_test)

layer_sizes = [200, 200]
assert len(layer_sizes) == len(num_samples)

graphsage = GraphSAGE(
        layer_sizes=layer_sizes, generator=train_gen, bias=True, dropout=0.2)

In [25]:
# Build the model and expose input and output sockets of graphsage model for link prediction via graphsage.build() method
x_inp, x_out = graphsage.build()

prediction = link_classification(
    output_dim=1, output_act="relu", edge_embedding_method='ip')(x_out)
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.binary_crossentropy,
        metrics=["acc"],)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [26]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           [(None, 200, 64)]    0                                            
__________________________________________________________________________________________________
input_27 (InputLayer)           [(None, 40000, 64)]  0                                            
__________________________________________________________________________________________________
input_29 (InputLayer)           [(None, 200, 64)]    0                                            
__________________________________________________________________________________________________
input_30 (InputLayer)           [(None, 40000, 64)]  0                                            
____________________________________________________________________________________________

In [ ]:
#init_train_metrics = model.evaluate_generator(train_gen)
#init_test_metrics = model.evaluate_generator(test_gen)
#
#print("\nTrain Set Metrics of the initial (untrained) model:")
#for name, val in zip(model.metrics_names, init_train_metrics):
#    print("\t{}: {:0.4f}".format(name, val))
#
#print("\nTest Set Metrics of the initial (untrained) model:")
#for name, val in zip(model.metrics_names, init_test_metrics):
#    print("\t{}: {:0.4f}".format(name, val))
    
history = model.fit_generator(
    train_gen,
    epochs=epochs,
    validation_data=test_gen,
    verbose=1
)

Epoch 1/5


 193/2553 [=>............................] - ETA: 79:19:50 - loss: 0.7471 - acc: 0.450 - ETA: 41:19:28 - loss: 0.6982 - acc: 0.525 - ETA: 28:17:32 - loss: 0.6251 - acc: 0.616 - ETA: 21:42:57 - loss: 0.6265 - acc: 0.650 - ETA: 17:59:44 - loss: 0.6235 - acc: 0.660 - ETA: 15:50:48 - loss: 0.6271 - acc: 0.641 - ETA: 14:26:39 - loss: 0.6237 - acc: 0.642 - ETA: 13:34:12 - loss: 0.6298 - acc: 0.631 - ETA: 12:52:37 - loss: 0.6265 - acc: 0.644 - ETA: 12:22:07 - loss: 0.6320 - acc: 0.635 - ETA: 11:44:15 - loss: 0.6304 - acc: 0.636 - ETA: 11:09:09 - loss: 0.6306 - acc: 0.641 - ETA: 10:41:50 - loss: 0.6216 - acc: 0.657 - ETA: 10:22:31 - loss: 0.6152 - acc: 0.660 - ETA: 10:08:37 - loss: 0.6163 - acc: 0.670 - ETA: 10:01:26 - loss: 0.6164 - acc: 0.675 - ETA: 9:57:47 - loss: 0.6104 - acc: 0.685 - ETA: 9:49:40 - loss: 0.6085 - acc: 0.68 - ETA: 9:38:34 - loss: 0.6092 - acc: 0.68 - ETA: 9:41:11 - loss: 0.6094 - acc: 0.68 - ETA: 9:34:12 - loss: 0.6070 - acc: 0.69 - ETA: 9:29:48 - loss: 0.6006 - acc: 0.69 

 778/2553 [========>.....................] - ETA: 3:53:16 - loss: 0.4934 - acc: 0.77 - ETA: 3:53:03 - loss: 0.4933 - acc: 0.77 - ETA: 3:52:50 - loss: 0.4932 - acc: 0.77 - ETA: 3:52:36 - loss: 0.4933 - acc: 0.77 - ETA: 3:52:23 - loss: 0.4934 - acc: 0.77 - ETA: 3:52:10 - loss: 0.4932 - acc: 0.77 - ETA: 3:51:57 - loss: 0.4930 - acc: 0.77 - ETA: 3:51:43 - loss: 0.4929 - acc: 0.77 - ETA: 3:51:29 - loss: 0.4925 - acc: 0.77 - ETA: 3:51:16 - loss: 0.4924 - acc: 0.77 - ETA: 3:51:03 - loss: 0.4923 - acc: 0.77 - ETA: 3:50:49 - loss: 0.4921 - acc: 0.77 - ETA: 3:50:36 - loss: 0.4922 - acc: 0.77 - ETA: 3:50:22 - loss: 0.4919 - acc: 0.77 - ETA: 3:50:09 - loss: 0.4917 - acc: 0.77 - ETA: 3:49:56 - loss: 0.4916 - acc: 0.77 - ETA: 3:49:43 - loss: 0.4919 - acc: 0.77 - ETA: 3:49:30 - loss: 0.4918 - acc: 0.77 - ETA: 3:49:17 - loss: 0.4917 - acc: 0.77 - ETA: 3:49:04 - loss: 0.4915 - acc: 0.77 - ETA: 3:48:51 - loss: 0.4918 - acc: 0.77 - ETA: 3:48:38 - loss: 0.4916 - acc: 0.77 - ETA: 3:48:25 - loss: 0.4915 - a

 973/2553 [==========>...................] - ETA: 3:18:06 - loss: 0.4881 - acc: 0.77 - ETA: 3:17:56 - loss: 0.4880 - acc: 0.77 - ETA: 3:17:46 - loss: 0.4881 - acc: 0.77 - ETA: 3:17:36 - loss: 0.4882 - acc: 0.77 - ETA: 3:17:27 - loss: 0.4881 - acc: 0.77 - ETA: 3:17:20 - loss: 0.4880 - acc: 0.77 - ETA: 3:17:10 - loss: 0.4881 - acc: 0.77 - ETA: 3:17:00 - loss: 0.4878 - acc: 0.77 - ETA: 3:16:49 - loss: 0.4878 - acc: 0.77 - ETA: 3:16:39 - loss: 0.4878 - acc: 0.77 - ETA: 3:16:29 - loss: 0.4878 - acc: 0.77 - ETA: 3:16:19 - loss: 0.4877 - acc: 0.77 - ETA: 3:16:09 - loss: 0.4878 - acc: 0.77 - ETA: 3:15:59 - loss: 0.4878 - acc: 0.77 - ETA: 3:15:48 - loss: 0.4879 - acc: 0.77 - ETA: 3:15:38 - loss: 0.4879 - acc: 0.77 - ETA: 3:15:28 - loss: 0.4880 - acc: 0.77 - ETA: 3:15:19 - loss: 0.4878 - acc: 0.77 - ETA: 3:15:08 - loss: 0.4876 - acc: 0.77 - ETA: 3:14:58 - loss: 0.4876 - acc: 0.77 - ETA: 3:14:48 - loss: 0.4875 - acc: 0.77 - ETA: 3:14:38 - loss: 0.4875 - acc: 0.77 - ETA: 3:14:28 - loss: 0.4874 - a

1168/2553 [============>.................] - ETA: 2:48:16 - loss: 0.4959 - acc: 0.77 - ETA: 2:48:08 - loss: 0.4961 - acc: 0.77 - ETA: 2:47:59 - loss: 0.4960 - acc: 0.77 - ETA: 2:47:50 - loss: 0.4960 - acc: 0.77 - ETA: 2:47:42 - loss: 0.4959 - acc: 0.77 - ETA: 2:47:33 - loss: 0.4958 - acc: 0.77 - ETA: 2:47:24 - loss: 0.4958 - acc: 0.77 - ETA: 2:47:16 - loss: 0.4958 - acc: 0.77 - ETA: 2:47:08 - loss: 0.4958 - acc: 0.77 - ETA: 2:47:00 - loss: 0.4957 - acc: 0.77 - ETA: 2:46:51 - loss: 0.4956 - acc: 0.77 - ETA: 2:46:43 - loss: 0.4955 - acc: 0.77 - ETA: 2:46:34 - loss: 0.4955 - acc: 0.77 - ETA: 2:46:26 - loss: 0.4954 - acc: 0.77 - ETA: 2:46:18 - loss: 0.4955 - acc: 0.77 - ETA: 2:46:09 - loss: 0.4956 - acc: 0.77 - ETA: 2:46:01 - loss: 0.4955 - acc: 0.77 - ETA: 2:45:52 - loss: 0.4955 - acc: 0.77 - ETA: 2:45:45 - loss: 0.4954 - acc: 0.77 - ETA: 2:45:38 - loss: 0.4955 - acc: 0.77 - ETA: 2:45:31 - loss: 0.4953 - acc: 0.77 - ETA: 2:45:22 - loss: 0.4954 - acc: 0.77 - ETA: 2:45:14 - loss: 0.4953 - a

...

1755/2553 [===================>..........] - ETA: 1:27:05 - loss: 0.4353 - acc: 0.80 - ETA: 1:26:59 - loss: 0.4353 - acc: 0.80 - ETA: 1:26:54 - loss: 0.4353 - acc: 0.80 - ETA: 1:26:48 - loss: 0.4352 - acc: 0.80 - ETA: 1:26:43 - loss: 0.4352 - acc: 0.80 - ETA: 1:26:38 - loss: 0.4352 - acc: 0.80 - ETA: 1:26:32 - loss: 0.4352 - acc: 0.80 - ETA: 1:26:27 - loss: 0.4352 - acc: 0.80 - ETA: 1:26:21 - loss: 0.4351 - acc: 0.80 - ETA: 1:26:16 - loss: 0.4351 - acc: 0.80 - ETA: 1:26:10 - loss: 0.4351 - acc: 0.80 - ETA: 1:26:05 - loss: 0.4352 - acc: 0.80 - ETA: 1:26:00 - loss: 0.4351 - acc: 0.80 - ETA: 1:25:54 - loss: 0.4351 - acc: 0.80 - ETA: 1:25:49 - loss: 0.4352 - acc: 0.80 - ETA: 1:25:44 - loss: 0.4352 - acc: 0.80 - ETA: 1:25:38 - loss: 0.4356 - acc: 0.80 - ETA: 1:25:33 - loss: 0.4355 - acc: 0.80 - ETA: 1:25:27 - loss: 0.4356 - acc: 0.80 - ETA: 1:25:22 - loss: 0.4355 - acc: 0.80 - ETA: 1:25:17 - loss: 0.4354 - acc: 0.80 - ETA: 1:25:11 - loss: 0.4354 - acc: 0.80 - ETA: 1:25:06 - loss: 0.4354 - a

1953/2553 [=====================>........] - ETA: 1:10:02 - loss: 0.4344 - acc: 0.80 - ETA: 1:09:57 - loss: 0.4344 - acc: 0.80 - ETA: 1:09:52 - loss: 0.4343 - acc: 0.80 - ETA: 1:09:46 - loss: 0.4345 - acc: 0.80 - ETA: 1:09:41 - loss: 0.4344 - acc: 0.80 - ETA: 1:09:36 - loss: 0.4344 - acc: 0.80 - ETA: 1:09:31 - loss: 0.4343 - acc: 0.80 - ETA: 1:09:25 - loss: 0.4342 - acc: 0.80 - ETA: 1:09:20 - loss: 0.4343 - acc: 0.80 - ETA: 1:09:15 - loss: 0.4343 - acc: 0.80 - ETA: 1:09:09 - loss: 0.4342 - acc: 0.80 - ETA: 1:09:04 - loss: 0.4342 - acc: 0.80 - ETA: 1:08:59 - loss: 0.4341 - acc: 0.80 - ETA: 1:08:54 - loss: 0.4342 - acc: 0.80 - ETA: 1:08:49 - loss: 0.4342 - acc: 0.80 - ETA: 1:08:44 - loss: 0.4342 - acc: 0.80 - ETA: 1:08:39 - loss: 0.4341 - acc: 0.80 - ETA: 1:08:34 - loss: 0.4341 - acc: 0.80 - ETA: 1:08:29 - loss: 0.4340 - acc: 0.80 - ETA: 1:08:24 - loss: 0.4339 - acc: 0.80 - ETA: 1:08:18 - loss: 0.4339 - acc: 0.80 - ETA: 1:08:13 - loss: 0.4338 - acc: 0.80 - ETA: 1:08:08 - loss: 0.4339 - a

2157/2553 [========================>.....] - ETA: 52:48 - loss: 0.4328 - acc: 0.80 - ETA: 52:43 - loss: 0.4327 - acc: 0.80 - ETA: 52:38 - loss: 0.4326 - acc: 0.80 - ETA: 52:33 - loss: 0.4326 - acc: 0.80 - ETA: 52:27 - loss: 0.4326 - acc: 0.80 - ETA: 52:22 - loss: 0.4326 - acc: 0.80 - ETA: 52:16 - loss: 0.4326 - acc: 0.80 - ETA: 52:11 - loss: 0.4325 - acc: 0.80 - ETA: 52:06 - loss: 0.4324 - acc: 0.80 - ETA: 52:01 - loss: 0.4323 - acc: 0.80 - ETA: 51:55 - loss: 0.4323 - acc: 0.80 - ETA: 51:50 - loss: 0.4323 - acc: 0.80 - ETA: 51:45 - loss: 0.4323 - acc: 0.80 - ETA: 51:39 - loss: 0.4324 - acc: 0.80 - ETA: 51:34 - loss: 0.4324 - acc: 0.80 - ETA: 51:29 - loss: 0.4324 - acc: 0.80 - ETA: 51:23 - loss: 0.4324 - acc: 0.80 - ETA: 51:18 - loss: 0.4324 - acc: 0.80 - ETA: 51:13 - loss: 0.4324 - acc: 0.80 - ETA: 51:07 - loss: 0.4324 - acc: 0.80 - ETA: 51:02 - loss: 0.4323 - acc: 0.80 - ETA: 50:57 - loss: 0.4323 - acc: 0.80 - ETA: 50:51 - loss: 0.4324 - acc: 0.80 - ETA: 50:46 - loss: 0.4324 - acc: 0.

2361/2553 [==========================>...] - ETA: 34:47 - loss: 0.4306 - acc: 0.80 - ETA: 34:42 - loss: 0.4305 - acc: 0.80 - ETA: 34:36 - loss: 0.4305 - acc: 0.80 - ETA: 34:31 - loss: 0.4304 - acc: 0.80 - ETA: 34:26 - loss: 0.4304 - acc: 0.80 - ETA: 34:20 - loss: 0.4304 - acc: 0.80 - ETA: 34:15 - loss: 0.4304 - acc: 0.80 - ETA: 34:10 - loss: 0.4303 - acc: 0.80 - ETA: 34:04 - loss: 0.4302 - acc: 0.80 - ETA: 33:59 - loss: 0.4302 - acc: 0.80 - ETA: 33:54 - loss: 0.4302 - acc: 0.80 - ETA: 33:48 - loss: 0.4302 - acc: 0.80 - ETA: 33:43 - loss: 0.4305 - acc: 0.80 - ETA: 33:38 - loss: 0.4305 - acc: 0.80 - ETA: 33:33 - loss: 0.4304 - acc: 0.80 - ETA: 33:27 - loss: 0.4303 - acc: 0.80 - ETA: 33:22 - loss: 0.4304 - acc: 0.80 - ETA: 33:17 - loss: 0.4304 - acc: 0.80 - ETA: 33:11 - loss: 0.4303 - acc: 0.80 - ETA: 33:06 - loss: 0.4303 - acc: 0.80 - ETA: 33:01 - loss: 0.4302 - acc: 0.80 - ETA: 32:55 - loss: 0.4302 - acc: 0.80 - ETA: 32:50 - loss: 0.4301 - acc: 0.80 - ETA: 32:45 - loss: 0.4301 - acc: 0.

2552/2553 [============================>.] - ETA: 16:51 - loss: 0.4317 - acc: 0.80 - ETA: 16:45 - loss: 0.4316 - acc: 0.80 - ETA: 16:40 - loss: 0.4316 - acc: 0.80 - ETA: 16:35 - loss: 0.4316 - acc: 0.80 - ETA: 16:29 - loss: 0.4316 - acc: 0.80 - ETA: 16:24 - loss: 0.4316 - acc: 0.80 - ETA: 16:19 - loss: 0.4316 - acc: 0.80 - ETA: 16:14 - loss: 0.4316 - acc: 0.80 - ETA: 16:08 - loss: 0.4315 - acc: 0.80 - ETA: 16:03 - loss: 0.4316 - acc: 0.80 - ETA: 15:58 - loss: 0.4316 - acc: 0.80 - ETA: 15:52 - loss: 0.4316 - acc: 0.80 - ETA: 15:47 - loss: 0.4315 - acc: 0.80 - ETA: 15:42 - loss: 0.4315 - acc: 0.80 - ETA: 15:36 - loss: 0.4315 - acc: 0.80 - ETA: 15:31 - loss: 0.4315 - acc: 0.80 - ETA: 15:26 - loss: 0.4315 - acc: 0.80 - ETA: 15:21 - loss: 0.4315 - acc: 0.80 - ETA: 15:15 - loss: 0.4315 - acc: 0.80 - ETA: 15:10 - loss: 0.4315 - acc: 0.80 - ETA: 15:05 - loss: 0.4315 - acc: 0.80 - ETA: 14:59 - loss: 0.4314 - acc: 0.80 - ETA: 14:54 - loss: 0.4314 - acc: 0.80 - ETA: 14:49 - loss: 0.4314 - acc: 0.